In [ ]:
"""
This file splits the dataset into training and test sets using an 80-20 split. It then produces our three algorithms 
(KNN, Logistic Regression, and Decision Tree) and performs cross-validation. You will have to fill in the code for producing 
the KNN models and Decision Tree models. The models are then compared based on precision and accuracy levels, with the highest
performing model identified at the end.
"""

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
from collections import Counter
import re
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score
from sklearn.utils import shuffle
from sklearn import datasets, neighbors
from mlxtend.plotting import plot_decision_regions

In [ ]:
# Read in CSV of cleaned/preprocessed data
# Change file path if necessary
df = pd.read_csv("preprocessed_data.csv")
df.head()

In [ ]:
# Extract labels and features
X = df["Preprocessed Text"]  # Preprocessed text data
y = df["Spam or Not Spam"]  # Target labels


In [ ]:
# Split into training (80%) and testing (20%) sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [ ]:
# Convert text to numerical representation using TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=5000)  # Can fine-tune max_features
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)


In [ ]:
# Hyperparameter ranges
knn_k_values = [3, 5, 10, 15]  # Different k values for KNN
decision_tree_depths = [5, 10, 15, 20]  # Different depths for Decision Tree
cross_validation_levels = [5, 10, 15]  # Different levels of cross-validation


In [ ]:
# Initialize models with tuning capability
models = {"Logistic Regression": LogisticRegression(max_iter=1000)}


In [ ]:
# In the spaces below, write the following code:

In [ ]:
# Use a for loop with the 'knn_k_values' variable to add the KNN models to the 'models' variable




In [ ]:
# Similarly, use a for loop with the 'decision_tree_depths' variable to add the decision tree 
# models to the 'models' variable




In [ ]:
# Train, evaluate, and perform cross-validation
results = []
for name, model in models.items():
    model.fit(X_train_tfidf, y_train)

    for cv in cross_validation_levels:
        cv_scores = cross_val_score(model, X_train_tfidf, y_train, cv=cv, scoring='accuracy')

        # Evaluate on test set
        y_pred = model.predict(X_test_tfidf)
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred, pos_label="Spam")

        # Save results to the list
        results.append({
            "Model": name,
            "Cross-Validation Folds": cv,
            "Test Accuracy": accuracy,
            "Precision": precision,
            "Cross-Validation Accuracy": np.mean(cv_scores)
        })


In [ ]:
# Convert results to DataFrame and save as CSV
results_df = pd.DataFrame(results)
results_df.to_csv("model_results.csv", index=False)

# Display results
for result in results:
    print(f"{result['Model']} (CV={result['Cross-Validation Folds']}):")
    print(f"  Test Accuracy: {result['Test Accuracy']:.4f}")
    print(f"  Precision: {result['Precision']:.4f}")
    print(f"  Cross-Validation Accuracy: {result['Cross-Validation Accuracy']:.4f}")
    print("-" * 40)

# Identify the best performing model for accuracy
best_model = max(results, key=lambda r: r["Test Accuracy"])
if best_model["Test Accuracy"] >= 0.95:
    print(f"{best_model['Model']} (CV={best_model['Cross-Validation Folds']}) meets/exceeds 95% accuracy.")
else:
    print(f"No model reached 95% accuracy. Further tuning is needed.")

# Identify the best performing model for precision
best_model = max(results, key=lambda r: r["Precision"])
if best_model["Precision"] >= 0.95:
    print(f"{best_model['Model']} (CV={best_model['Cross-Validation Folds']}) meets/exceeds 95% precision.")
else:
    print(f"No model reached 95% precision. Further tuning is needed.")

In [ ]:
# Load the results CSV
results_df = pd.read_csv("model_results.csv")

# Set style of graphs
sns.set_style("whitegrid")

# Grouped Bar Chart (for comparing all three variables between each of the models)
plt.figure(figsize=(12, 6))
metrics = ["Test Accuracy", "Precision", "Cross-Validation Accuracy"]
results_df_melted = results_df.melt(id_vars=["Model"], value_vars=metrics, var_name="Metric", value_name="Score")

sns.barplot(data=results_df_melted, x="Model", y="Score", hue="Metric", palette="coolwarm")
plt.xticks(rotation=45, ha="right")
plt.title("Model Comparison: Accuracy, Precision, and Cross-Validation")
plt.legend(title="Metric")
plt.show()

# Line Plot (for performance trends across models)
plt.figure(figsize=(12, 6))
for metric in metrics:
    sns.lineplot(data=results_df, x="Model", y=metric, marker="o", label=metric)

plt.xticks(rotation=45, ha="right")
plt.title("Performance Trends Across Models")
plt.ylabel("Score")
plt.legend(title="Metric")
plt.show()

# Box Plot (showing the distribution of scores)
plt.figure(figsize=(10, 6))
sns.boxplot(data=results_df_melted, x="Metric", y="Score", hue="Metric", palette="coolwarm", dodge=False)
plt.title("Performance Variation Across Models")
plt.show()

In [ ]:
from scipy.stats import pearsonr

# Load results
results_df = pd.read_csv("model_results.csv")

# Encode model names numerically
results_df["Model_Num"] = results_df["Model"].astype("category").cat.codes  #Assigns numerical values to models

# Compute Pearson Correlation
correlations = {}
for metric in ["Test Accuracy", "Precision", "Cross-Validation Accuracy"]:
    r_value, p_value = pearsonr(results_df["Model_Num"], results_df[metric])
    correlations[metric] = {"r-value": r_value, "p-value": p_value}

# Convert to DataFrame
corr_df = pd.DataFrame.from_dict(correlations, orient="index")
print("Pearson Correlation Coefficients:")
print(corr_df)